# DM - Práctica Text Mining
___

- Universidad Carlos III de Madrid, 4º Curso
- Asignatura: Métodos estadísticos en minería de datos
- Número de grupo: no se que numero somos hahhahahhahahah
- Integrantes: Elena Tran Yang, Marta Madella, Jin Wang y Sabrina Torres Román

In [ ]:
holasafv

In [ ]:
# Cargar dataset
ds = load_dataset("Iker/NoticIA")

df = pd.concat([
ds['train'].to_pandas(),
ds['validation'].to_pandas(),
ds['test'].to_pandas()
], ignore_index=True)